In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_capillary',
    'customer_capillary_daily',
    proc_date,
    'temp_customer_capillary_daily'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_capillary.customer_capillary_daily
(
    user_id STRING,
    merchant_customer_id STRING,
    mobile STRING,
    gender STRING,
    binding_date STRING,
    dob STRING,
    reg_source STRING,
    fullname STRING,
    store_id STRING,
    bu_id STRING,
    calday DATE,
    returned_points DOUBLE,
    customer_id STRING,
    masan_referral_code STRING,
    staff_id STRING,
    merchant_store_id STRING,
    mobile_enroll STRING,
    marketing_noti STRING,
    pilot_store_id STRING,
    pos_enroll STRING,
    onboarding_store_id STRING,
    mobile_enroll_date STRING,
    mobile_noti STRING,
    policy_items STRING,
    device STRING,
    pilot_group STRING,
    loyalty_points DOUBLE,
    lifetime_points DOUBLE,
    redeemed_points DOUBLE,
    expired_points DOUBLE,
    proc_date DATE,
    file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_capillary.customer_capillary_daily
    WHERE proc_date = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_capillary.customer_capillary_daily
SELECT user_user_id as user_id
        ,merchant_customer_id
        ,mobile
        ,gender
        ,binding_date
        ,dob
        ,reg_source
        ,fullname
        ,store_id
        ,bu_id
        ,calday
        ,returned_points
        ,customer_id
        ,masan_referral_code
        ,IFNULL(staff_id,"")
        ,merchant_store_id
        ,mobile_enroll
        ,marketing_noti
        ,pilot_store_id
        ,pos_enroll
        ,onboarding_store_id
        ,mobile_enroll_date
        ,mobile_noti
        ,policy_items
        ,device
        ,pilot_group
        ,loyalty_points
        ,lifetime_points
        ,redeemed_points
        ,expired_points
        ,proc_date
        ,file_creation_ts
FROM temp_customer_capillary_daily;
""")